In [8]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from bs4 import BeautifulSoup

In [18]:
def init_browser():
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)
    return browser

In [21]:
def scrape():
    browser = init_browser()

    url = "https://mars.nasa.gov/news/"
    browser.visit(url)

    html = browser.html
    soup = BeautifulSoup(html, "html.parser")

    news_title = soup.find("div", class_="content_title").get_text()
    news_p = soup.find("div", class_="article_teaser_body").get_text()
    
    print(news_title)

    return (news_title, news_p)

In [22]:
scrape()

MarCO Makes Space for Small Explorers


('MarCO Makes Space for Small Explorers',
 'A pair of NASA CubeSats flying to Mars are opening a new frontier for small spacecraft.')

In [28]:
def scrape_2():
    browser = init_browser()

    url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    browser.visit(url)

    browser.click_link_by_partial_text('FULL IMAGE')
    browser.is_element_present_by_text("more info", wait_time=0.5)
    browser.click_link_by_partial_text('more info')

    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    fig_1 = soup.find("figure", class_="lede")
    img_link = fig_1.a['href']
    featured_image_url = "https://www.jpl.nasa.gov" + img_link
    
    print(featured_image_url)

    return (featured_image_url)
scrape_2()

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17171_hires.jpg


'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17171_hires.jpg'

In [33]:
browser = init_browser()

url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")
mars_weather = soup.find("div", class_="js-tweet-text-container").p.text
print(mars_weather)

Sol 2171 (2018-09-14), high -12C/10F, low -65C/-84F, pressure at 8.79 hPa, daylight 05:43-17:59


In [37]:
import pandas as pd
url = "http://space-facts.com/mars/"
tables = pd.read_html(url)
tables[0]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [48]:
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
links = soup.find_all("a", class_="itemLink product-item")
print(links[0]['href'])

/search/map/Mars/Viking/cerberus_enhanced


In [ ]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
# Define database and collection
db = client.craigslist_db
collection = db.items

In [2]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [4]:
results = soup.find('div', class_='content_title')
print(results)

<div class="content_title">
<a href="/news/8348/opportunity-hunkers-down-during-dust-storm/">
Opportunity Hunkers Down During Dust Storm
</a>
</div>


In [ ]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
# results = soup.find_all('li', class_='content_title')

news_title = "NASA's Next Mars Mission to Investigate Interior of Red Planet"

news_p = "Preparation of NASA's next spacecraft to Mars, InSight, has ramped up this summer, on course for launch next May from Vandenberg Air Force Base in central California -- the first interplanetary launch in history from America's West Coast."


# Loop through returned results
for result in results:
    # Error handling
    try:
        # Identify and return title of listing
        title = result.find('a', class_='result-title').text
        # Identify and return price of listing
        price = result.a.span.text
        # Identify and return link to listing
        link = result.a['href']

        # Run only if title, price, and link are available
        if (title and price and link):
            # Print results
            print('-------------')
            print(title)
            print(price)
            print(link)

            # Dictionary to be inserted as a MongoDB document
            post = {
                'title': title,
                'price': price,
                'url': link
            }

            collection.insert_one(post)

    except Exception as e:
        print(e)

In [ ]:
# Display items in MongoDB collection
listings = db.items.find()

for listing in listings:
    print(listing)